# Kurulum ve Bağımlılıklar
Bu bölüm gerekli Python paketlerinin kurulumu içindir. Colab/Kaggle ortamında tek seferlik çalıştırınız.

In [ ]:
!pip install -U agno duckduckgo-search ollama lancedb unstructured tantivy pylance transformers soundfile ipywidgets

# Kütüphanelerin Yüklenmesi
Gerekli tüm Python kütüphaneleri bu bölümde içe aktarılır. Bu kütüphaneler, Agno framework'ü, Hugging Face Transformers, Ollama entegrasyonu ve çeşitli yardımcı araçları içerir.

In [ ]:
from __future__ import annotations

# Standart kütüphaneler
import os
import sys
import time
import json
import uuid
import random
from datetime import datetime, timedelta, date
from typing import Optional, List, Literal, TypedDict
import base64
import subprocess # Ollama sunucu kontrolü için
import pandas as pd # Veri çerçeveleri için
import glob # Dosya yollarını bulmak için

# Genel kütüphaneler
import requests
import numpy as np
import torch
import scipy.io.wavfile as wavfile

# Transformers / Audio / Notebook yardımcıları
from transformers import pipeline, VitsModel, AutoTokenizer
from IPython.display import Audio, display, Javascript, clear_output
import ipywidgets as widgets

# Ortam değişkenleri
from dotenv import load_dotenv

# Colab (opsiyonel, yerel ortamda hata vermesin diye korumalı)
try:
    from google.colab import output as colab_output  # type: ignore
except Exception:
    colab_output = None  # Colab dışı ortamda kullanılmaz

# Agno / Ollama ekosistemi
from ollama import chat
from agno.tools import tool
from agno.agent import Agent
from agno.team.team import Team
from agno.models.ollama import Ollama
from agno.embedder.ollama import OllamaEmbedder
from agno.models.huggingface import HuggingFace
from agno.tools.yfinance import YFinanceTools
from agno.knowledge.url import UrlKnowledge
from agno.knowledge.markdown import MarkdownKnowledgeBase
from agno.storage.sqlite import SqliteStorage
from agno.vectordb.lancedb import LanceDb, SearchType
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.memory import Memory
from agno.tools.reasoning import ReasoningTools

# Agno değerlendirme modülleri
from agno.eval.accuracy import AccuracyEval, AccuracyResult
from agno.eval.performance import PerformanceEval
from agno.eval.reliability import ReliabilityEval, ReliabilityResult

# STT - Whisper Hazırlık
Whisper tabanlı konuşma tanıma (Speech-to-Text) için pipeline ve yardımcı fonksiyon.

In [ ]:

stt = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3")

# Speech-to-Text (Whisper)
def speech_to_text(audio_file):
    result = stt(audio_file, return_timestamps=True)
    return result["text"]

# TTS - VITS Model Hazırlık
VITS tabanlı metin okuma (Text-to-Speech) için model ve tokenizer yüklemesi yapılır. Bu model, agent'ın metin yanıtlarını sesli çıktılara dönüştürmek için kullanılır.

In [ ]:
model = VitsModel.from_pretrained("facebook/mms-tts-tur").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("facebook/mms-tts-tur")

def text_to_speech(text, output_file="output.wav"):
    """Metni VITS modeli kullanarak sese dönüştürür ve bir Audio nesnesi döndürür."""
    # Metni model için uygun formata çevir
    inputs = tokenizer(text, return_tensors="pt")
    # Konuşma dalga formunu üret
    with torch.no_grad():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
        output = model(**inputs).waveform.cpu()
    return Audio(output, rate=model.config.sampling_rate)

# Ollama Sunucusu Kurulum ve Model Çekme
Yerel Ollama servisini başlatma ve gerekli LLM/embedding modellerini çekme adımları.

In [ ]:
# 0) Ollama'yı kur (oturum başına bir kez)
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=True)

# 1) Ollama sunucusunu arka planda başlat
srv = subprocess.Popen("ollama serve", shell=True)

# 2) API hazır olana kadar bekle
for _ in range(60):
    try:
        r = requests.get("http://127.0.0.1:11434/api/version", timeout=1)
        if r.ok: break
    except Exception:
        time.sleep(1)

# 3) Gerekli modelleri çek (LLM ve embedding)
subprocess.run("ollama pull qwen2.5:7b-instruct", shell=True, check=True)
subprocess.run("ollama pull bge-m3", shell=True, check=True)

# Mock Veritabanları ve Fonksiyonları
Bu bölümde, müşteri hizmetleri senaryolarını simüle eden tüm mock veritabanları ve bu verilere erişen @tool dekore edilmiş fonksiyonlar tanımlanır. Bu bölüm, agent'ın kullanacağı tüm araçların arka uç mantığını içerir.

In [ ]:
# ----------------------------
# Kullanıcı Bilgileri
# ----------------------------
_user_store = {
    "12345": {"user_id": "12345", "name": "Ayşe", "birth_date": "1985-07-12", "current_package": "Gold"},
    "67890": {"user_id": "67890", "name": "Mehmet", "birth_date": "1990-05-25", "current_package": "Basic"},
    "54321": {"user_id": "54321", "name": "Zeynep", "birth_date": "1978-11-03", "current_package": "Premium"},
    "11111": {"user_id": "11111", "name": "Ali",    "birth_date": "1988-01-09", "current_package": "Student"},
    "22222": {"user_id": "22222", "name": "Elif",   "birth_date": "1995-03-14", "current_package": "Gold"},
    "33333": {"user_id": "33333", "name": "Selim",  "birth_date": "1982-09-28", "current_package": "Family"},
    "44444": {"user_id": "44444", "name": "Fatma",  "birth_date": "1976-02-02", "current_package": "Basic"},
    "55555": {"user_id": "55555", "name": "Deniz",  "birth_date": "2001-12-21", "current_package": "Unlimited"},
    "66666": {"user_id": "66666", "name": "Ece",    "birth_date": "1999-06-06", "current_package": "Premium"},
    "77777": {"user_id": "77777", "name": "Murat",  "birth_date": "1987-04-18", "current_package": "Platinum"},
    "88888": {"user_id": "88888", "name": "Can",    "birth_date": "1993-10-10", "current_package": "Basic"},
    "99999": {"user_id": "99999", "name": "Hakan",  "birth_date": "1980-08-30", "current_package": "Gold"},
}

# ----------------------------
# Paket Bilgileri
# ----------------------------
_package_store = [
    {"id": "Basic",   "price": 10.0, "description": "Basic subscription"},
    {"id": "Gold",    "price": 20.0, "description": "Gold tier with extra features"},
    {"id": "Premium", "price": 30.0, "description": "All-inclusive Premium package"},
    {"id": "Student",   "price": 8.0,  "description": "Öğrenciler için indirimli plan"},
    {"id": "Family",    "price": 45.0, "description": "4 hatta kadar aile paketi"},
    {"id": "Platinum",  "price": 40.0, "description": "VIP destekli yüksek performanslı plan"},
    {"id": "Unlimited", "price": 55.0, "description": "Sınırsız veri, arama ve SMS"},
]

# Özellik haritası (isteğe bağlı yardımcı)
_package_features_store = {
    "Basic":     ["2 GB veri", "200 dk", "100 SMS"],
    "Gold":      ["10 GB veri", "1000 dk", "Sınırsız SMS"],
    "Premium":   ["50 GB veri", "Sınırsız dk", "Sınırsız SMS", "AB Roaming"],
    "Student":   ["5 GB veri", "500 dk", "200 SMS", "Öğrenci indirimi"],
    "Family":    ["Paylaşımlı 60 GB", "Sınırsız dk", "Sınırsız SMS", "4 hat"],
    "Platinum":  ["100 GB veri", "Sınırsız dk", "Sınırsız SMS", "Öncelikli destek"],
    "Unlimited": ["Sınırsız veri", "Sınırsız dk", "Sınırsız SMS", "5G öncelik"],
}

# ----------------------------
# Faturalandırma / Faturalar
# ----------------------------
class Invoice(TypedDict, total=False):
    invoice_id: str
    user_id: str
    date: str          # ISO date
    due_date: str      # ISO date
    amount: float
    currency: str
    status: Literal["paid", "due", "overdue", "refunded"]
    items: list[dict]

_invoice_store: dict[str, list[Invoice]] = {}

def _seed_invoices():
    today = date.today()
    for uid in _user_store.keys():
        invoices = []
        for m in range(1, 7):  # last 6 months
            inv_date = date(today.year, max(1, today.month - 6 + m), 5)
            amount = next((p["price"] for p in _package_store if p["id"] == _user_store[uid]["current_package"]), 20.0)
            status = random.choice(["paid", "paid", "paid", "due"])  # mostly paid
            invoices.append({
                "invoice_id": str(uuid.uuid4()),
                "user_id": uid,
                "date": inv_date.isoformat(),
                "due_date": (inv_date + timedelta(days=10)).isoformat(),
                "amount": round(amount, 2),
                "currency": "EUR",
                "status": status,
                "items": [{"description": f"Subscription - {_user_store[uid]['current_package']}", "price": amount}],
            })
        _invoice_store[uid] = invoices

_seed_invoices()

# ----------------------------
# Kullanım Metrikleri (aylık)
# ----------------------------
class Usage(TypedDict, total=False):
    month: str  # YYYY-MM
    data_gb: float
    minutes: int
    sms: int

_usage_store: dict[str, list[Usage]] = {}

def _seed_usage():
    today = date.today()
    for uid in _user_store.keys():
        rows: list[Usage] = []
        for m in range(1, 7):
            month_date = date(today.year, max(1, today.month - 6 + m), 1)
            rows.append({
                "month": month_date.strftime("%Y-%m"),
                "data_gb": round(random.uniform(1.0, 80.0), 2),
                "minutes": random.randint(50, 3000),
                "sms": random.randint(0, 1000),
            })
        _usage_store[uid] = rows

_seed_usage()

# ----------------------------
# Ödeme Yöntemleri
# ----------------------------
class PaymentMethod(TypedDict, total=False):
    method_id: str
    type: Literal["card", "iban"]
    brand: Optional[str]
    last4: Optional[str]
    exp: Optional[str]
    iban: Optional[str]
    default: bool

_payment_method_store: dict[str, list[PaymentMethod]] = {
    "12345": [{"method_id": "pm_ayse_card", "type": "card", "brand": "VISA", "last4": "4242", "exp": "12/26", "default": True}],
    "67890": [{"method_id": "pm_meh_iban", "type": "iban", "iban": "TR12 3456 7890 1234 5678 90", "default": True}],
}

# ----------------------------
# Destek Biletleri
# ----------------------------
class TicketMessage(TypedDict, total=False):
    at: str
    sender: Literal["user", "agent"]
    text: str

class Ticket(TypedDict, total=False):
    ticket_id: str
    user_id: str
    subject: str
    status: Literal["open", "pending", "resolved", "closed"]
    priority: Literal["low", "normal", "high"]
    created_at: str
    messages: list[TicketMessage]

_ticket_store: dict[str, Ticket] = {}

# ----------------------------
# Promosyonlar
# ----------------------------
class Promo(TypedDict, total=False):
    code: str
    type: Literal["percent", "fixed"]
    value: float
    valid_from: str
    valid_to: str
    applies_to: list[str]  # package ids
    max_uses: int
    used_count: int

_promo_store: list[Promo] = [
    {"code": "WELCOME10", "type": "percent", "value": 10.0, "valid_from": "2025-01-01", "valid_to": "2026-01-01",
     "applies_to": ["Basic", "Gold", "Premium"], "max_uses": 10000, "used_count": 1234},
    {"code": "STUDENT5",  "type": "fixed",   "value": 5.0,  "valid_from": "2025-03-01", "valid_to": "2026-03-01",
     "applies_to": ["Student"], "max_uses": 5000, "used_count": 450},
]

# ----------------------------
# Sözleşmeler
# ----------------------------
class Contract(TypedDict, total=False):
    user_id: str
    start_date: str
    end_date: Optional[str]
    termination_fee: float

_contract_store: dict[str, Contract] = {
    "12345": {"user_id": "12345", "start_date": "2024-07-15", "end_date": "2026-07-15", "termination_fee": 60.0},
    "67890": {"user_id": "67890", "start_date": "2023-05-25", "end_date": None,         "termination_fee": 0.0},
}

# ----------------------------
# Kesintiler (bölge/şehir)
# ----------------------------
class Outage(TypedDict, total=False):
    id: str
    city: str
    started_at: str
    status: Literal["investigating", "identified", "monitoring", "resolved"]
    eta: Optional[str]
    note: str

_outage_store: list[Outage] = [
    {"id": "out_istanbul_1", "city": "Istanbul", "started_at": "2025-08-13T06:30:00Z", "status": "identified",
     "eta": "2025-08-14T12:00:00Z", "note": "Kadıköy'de 5G düğüm kesintisi."},
    {"id": "out_ankara_1",   "city": "Ankara",   "started_at": "2025-08-12T20:10:00Z", "status": "monitoring",
     "eta": None, "note": "Kısa süreli paket kaybı gözlemlendi; performans iyileşiyor."},
]

# ----------------------------
# OTP depolama (kimlik doğrulama simülasyonu)
# ----------------------------
_otp_store: dict[str, str] = {}  # user_id -> last sent OTP (plain text for mock)

# ----------------------------
# Superonline Müşteri ve Hat Verileri
# ----------------------------
_user_store_superonline = {
    "10000001": {
        "customer_no": "10000001",
        "phone": "05321234567",
        "name": "Ali Yılmaz",
        "city": "Ankara",
        "service_status": "Active",   # Active / Suspended
        "connection_type": "Fiber",
        "plan": "100 Mbps Limitsiz"
    },
    "10000002": {
        "customer_no": "10000002",
        "phone": "05419876543",
        "name": "Fatma Demir",
        "city": "Istanbul",
        "service_status": "Suspended",
        "connection_type": "ADSL",
        "plan": "24 Mbps Limitsiz"
    }
}

_area_outage_store = {
    "Ankara": False,
    "Istanbul": True
}

_line_test_results = {
    "10000001": {
        "dsl_port_status": "UP",
        "ping_ms": 15,
        "sync_status": "OK",
        "ip_session": "Active"
    },
    "10000002": {
        "dsl_port_status": "DOWN",
        "ping_ms": None,
        "sync_status": "No Sync",
        "ip_session": "Inactive"
    }
}

_fault_ticket_store = []

# Modem Status Data
_modem_status_store = {
    "10000001": {"powered_on": True, "dsl_light": True},
    "10000002": {"powered_on": True, "dsl_light": False}
}

_technician_schedule_store = [] # Teknik destek için randevu mağazası

_invoice_dispute_store = [] # Fatura itirazları için

_addon_store = [
    {"id": "ExtraData1GB", "price": 5.0, "description": "Ek 1GB mobil internet", "duration_days": 30},
    {"id": "IntlCalls100", "price": 15.0, "description": "100 dakika uluslararası arama", "duration_days": 30},
    {"id": "StreamingPack", "price": 12.0, "description": "Sınırsız video platform erişimi", "duration_days": 30}
]

_user_addon_store = []


# ----------------------------
# Mock Fonksiyonlar / Tools
# ----------------------------

@tool
def mock_get_user_info(user_id: str) -> dict:
    """
    Fetch user profile by user_id.
    Returns a dict containing user_id, name, and current_package.
    Raises KeyError if user not found.
    """
    if user_id not in _user_store:
        raise KeyError(f"No user found with id={user_id}")
    return dict(_user_store[user_id])

@tool
def mock_get_available_packages() -> list[dict]:
    """
    List all subscription packages.
    Returns a list of dicts with keys: id, price, description.
    """
    return [dict(pkg) for pkg in _package_store]

@tool
def mock_initiate_package_change(user_id: str, new_pkg_id: str) -> bool:
    """
    Change a user's package.
    If user_id and new_pkg_id exist, update and return True.
    Otherwise return False.
    """
    user = _user_store.get(user_id)
    if not user:
        return False

    pkg_ids = {pkg["id"] for pkg in _package_store}
    if new_pkg_id not in pkg_ids:
        return False

    user["current_package"] = new_pkg_id
    return True

@tool
def mock_get_package_features(pkg_id: str) -> list[str]:
    """Return a human-readable feature list for a given package id."""
    return _package_features_store.get(pkg_id, [])

@tool
def mock_get_billing_history(user_id: str, limit: int = 12) -> list[Invoice]:
    """Return the user's recent invoices (most recent first)."""
    if user_id not in _invoice_store:
        return []
    return list(sorted(_invoice_store[user_id], key=lambda r: r["date"], reverse=True))[:limit]

@tool
def mock_get_latest_invoice(user_id: str) -> Optional[Invoice]:
    """Return the most recent invoice for a user, if any."""
    hist = mock_get_billing_history.invoke({"user_id": user_id, "limit": 1})  # type: ignore[attr-defined]
    return hist[0] if hist else None

@tool
def mock_pay_invoice(user_id: str, invoice_id: str, method_id: Optional[str] = None) -> bool:
    """
    Mark an invoice as paid if it belongs to the user and is due/overdue.
    Returns True on success.
    """
    for inv in _invoice_store.get(user_id, []):
        if inv["invoice_id"] == invoice_id and inv["status"] in {"due", "overdue"}:
            inv["status"] = "paid"
            return True
    return False

@tool
def mock_list_payment_methods(user_id: str) -> list[PaymentMethod]:
    """List user's saved payment methods, if any."""
    return list(_payment_method_store.get(user_id, []))

@tool
def mock_add_payment_method(user_id: str, method: dict) -> str:
    """
    Add a payment method (dict). Returns new method_id.
    Example method: {"type":"card","brand":"VISA","last4":"1234","exp":"10/28","default":True}
    """
    method_id = f"pm_{uuid.uuid4().hex[:8]}"
    method = {"method_id": method_id, "default": False, **method}
    _payment_method_store.setdefault(user_id, []).append(method)
    # If it's the first method, set default=True
    if sum(1 for _ in _payment_method_store[user_id]) == 1:
        _payment_method_store[user_id][0]["default"] = True
    return method_id

@tool
def mock_get_usage(user_id: str, month: Optional[str] = None) -> list[Usage] | Usage | None:
    """
    Get usage. If month='YYYY-MM' is provided, return that month only; else return last 6 months.
    """
    rows = _usage_store.get(user_id, [])
    if not rows:
        return None
    if month:
        for r in rows:
            if r["month"] == month:
                return r
        return None
    return rows

@tool
def mock_open_support_ticket(user_id: str, subject: str, message: str, priority: Literal["low","normal","high"]="normal") -> str:
    """
    Open a support ticket and return ticket_id.
    """
    tid = f"tkt_{uuid.uuid4().hex[:10]}"
    _ticket_store[tid] = {
        "ticket_id": tid,
        "user_id": user_id,
        "subject": subject,
        "status": "open",
        "priority": priority,
        "created_at": datetime.utcnow().isoformat() + "Z",
        "messages": [{"at": datetime.utcnow().isoformat() + "Z", "sender": "user", "text": message}],
    }
    return tid

@tool
def mock_get_ticket_status(ticket_id: str) -> Optional[str]:
    """Return status for a support ticket id."""
    t = _ticket_store.get(ticket_id)
    return t["status"] if t else None

@tool
def mock_reply_to_ticket(ticket_id: str, message: str, sender: Literal["user","agent"]="agent") -> bool:
    """Append a message to the ticket conversation."""
    t = _ticket_store.get(ticket_id)
    if not t:
        return False
    t["messages"].append({"at": datetime.utcnow().isoformat() + "Z", "sender": sender, "text": message})
    # naive state machine: agent reply moves from "open" -> "pending"
    if sender == "agent" and t["status"] == "open":
        t["status"] = "pending"
    return True

@tool
def mock_cancel_subscription(user_id: str, effective_date: Optional[str] = None) -> dict:
    """
    Cancel a subscription (mock). Returns summary with potential termination fee if under contract.
    """
    contract = _contract_store.get(user_id)
    fee = 0.0
    today_iso = date.today().isoformat()
    when = effective_date or today_iso
    if contract and contract.get("end_date"):
        # If today < end_date, apply fee
        end = date.fromisoformat(contract["end_date"])
        today_d = date.fromisoformat(when)
        if today_d < end:
            fee = float(contract.get("termination_fee", 0.0))
    return {"user_id": user_id, "effective_date": when, "termination_fee": fee, "status": "scheduled"}

@tool
def mock_calculate_proration(user_id: str, new_pkg_id: str) -> dict:
    """
    Rough proration estimate from current package to new_pkg_id for current billing cycle.
    """
    cur_pkg = _user_store.get(user_id, {}).get("current_package")
    if not cur_pkg:
        return {"ok": False, "reason": "user_not_found"}
    prices = {p["id"]: p["price"] for p in _package_store}
    if new_pkg_id not in prices:
        return {"ok": False, "reason": "pkg_not_found"}
    delta = prices[new_pkg_id] - prices.get(cur_pkg, 0.0)
    # naive proration: 50% of delta this cycle
    return {"ok": True, "from": cur_pkg, "to": new_pkg_id, "estimated_proration": round(delta * 0.5, 2), "currency": "EUR"}

@tool
def mock_list_promotions() -> list[Promo]:
    """List active promotions."""
    today = date.today().isoformat()
    active = []
    for p in _promo_store:
        if p["valid_from"] <= today <= p["valid_to"]:
            active.append(dict(p))
    return active

@tool
def mock_apply_promo(user_id: str, code: str) -> dict:
    """
    Validate and 'apply' a promo code (no real side-effects; returns the discount if valid).
    """
    code = code.upper().strip()
    user_pkg = _user_store.get(user_id, {}).get("current_package")
    today = date.today().isoformat()
    for p in _promo_store:
        if p["code"] == code and p["valid_from"] <= today <= p["valid_to"]:
            if user_pkg in p["applies_to"]:
                discount = p["value"] if p["type"] == "fixed" else None
                if p["type"] == "percent":
                    price = next((x["price"] for x in _package_store if x["id"] == user_pkg), 0.0)
                    discount = round(price * (p["value"] / 100.0), 2)
                return {"ok": True, "code": code, "discount": discount, "currency": "EUR"}
            return {"ok": False, "reason": "not_applicable_to_package"}
    return {"ok": False, "reason": "invalid_or_expired"}

@tool
def mock_recommend_package(user_id: str) -> dict:
    """
    Simple heuristic recommender using recent usage to suggest a package.
    """
    usage = _usage_store.get(user_id, [])
    if not usage:
        return {"ok": False, "reason": "no_usage"}
    avg_gb = sum(u["data_gb"] for u in usage) / len(usage)
    avg_min = sum(u["minutes"] for u in usage) / len(usage)
    # Heuristic thresholds
    if avg_gb > 60 or avg_min > 2500:
        rec = "Unlimited"
    elif avg_gb > 25 or avg_min > 1500:
        rec = "Platinum"
    elif avg_gb > 8 or avg_min > 500:
        rec = "Premium"
    else:
        rec = "Gold"
    price = next((p["price"] for p in _package_store if p["id"] == rec), None)
    return {"ok": True, "recommended": rec, "reason": f"avg_gb={avg_gb:.1f}, avg_min={int(avg_min)}", "price": price}

@tool
def mock_verify_identity(user_id: str, birth_date: str) -> bool:
    """
    Naive KBA: check if provided birth_date matches what's on file.
    """
    user = _user_store.get(user_id)
    return bool(user and user.get("birth_date") == birth_date)

@tool
def mock_send_otp(user_id: str) -> str:
    """
    Send an OTP (mock). Returns the OTP so tests can validate it.
    """
    code = f"{random.randint(100000, 999999)}"
    _otp_store[user_id] = code
    return code

@tool
def mock_validate_otp(user_id: str, code: str) -> bool:
    """Validate last OTP sent to the user."""
    return _otp_store.get(user_id) == code

@tool
def mock_get_outages(city: Optional[str] = None) -> list[Outage]:
    """
    List current network incidents; filter by city if provided.
    """
    if city:
        return [o for o in _outage_store if o["city"].lower() == city.lower()]
    return list(_outage_store)

@tool
def mock_upgrade_with_effective_date(user_id: str, new_pkg_id: str, effective_date: Optional[str] = None) -> dict:
    """
    Schedule a package change on an effective date (defaults to today). Returns summary.
    """
    if user_id not in _user_store:
        return {"ok": False, "reason": "user_not_found"}
    valid_ids = {p["id"] for p in _package_store}
    if new_pkg_id not in valid_ids:
        return {"ok": False, "reason": "pkg_not_found"}
    eff = effective_date or date.today().isoformat()
    # In a real system we would persist a future change; here we just echo summary.
    return {"ok": True, "user_id": user_id, "from": _user_store[user_id]["current_package"], "to": new_pkg_id, "effective_date": eff}

@tool
def mock_refund_last_charge(user_id: str, reason: str) -> dict:
    """
    Mark the last paid invoice as refunded (mock).
    """
    invoices = sorted(_invoice_store.get(user_id, []), key=lambda r: r["date"], reverse=True)
    for inv in invoices:
        if inv["status"] == "paid":
            inv["status"] = "refunded"
            return {"ok": True, "invoice_id": inv["invoice_id"], "amount": inv["amount"], "currency": inv["currency"]}
    return {"ok": False, "reason": "no_paid_invoice"}

@tool
def mock_get_latest_invoice_original(user_id: str) -> dict: # Eski fonksiyon adı
    """
    Fetch the latest invoice for a user.
    Returns a dict with invoice details or error if not found.
    """
    invoices = _invoice_store.get(user_id)
    if not invoices:
        return {"error": "Fatura bulunamadı"}
    return dict(invoices[-1])  # Last invoice

@tool
def mock_register_invoice_dispute(user_id: str, invoice_id: str, reason: str) -> bool:
    """
    Record a dispute for a specific invoice.
    Returns True if recorded, False if invoice not found.
    """
    invoices = _invoice_store.get(user_id, [])
    if not any(inv["invoice_id"] == invoice_id for inv in invoices):
        return False

    _invoice_dispute_store.append({
        "user_id": user_id,
        "invoice_id": invoice_id,
        "reason": reason,
        "status": "Pending Review"
    })
    return True

@tool
def mock_check_service_status_original(user_id: str) -> dict: # Eski fonksiyon adı
    """
    Check if the user's internet service is active or faulty.
    """
    status = _service_status_store.get(user_id)
    if not status:
        return {"error": "Hizmet bilgisi bulunamadı"}
    return dict(status)

@tool
def mock_check_area_outage_original(city: str) -> bool: # Eski fonksiyon adı
    """
    Check if there's a known outage in the user's city.
    Returns True if outage exists, False otherwise.
    """
    return _area_outage_store.get(city, False)

@tool
def mock_schedule_technician(user_id: str, date: str) -> bool:
    """
    Schedule a technician appointment.
    """
    _technician_schedule_store.append({
        "user_id": user_id,
        "date": date,
        "status": "Scheduled"
    })
    return True

@tool
def mock_get_available_addons(user_id: str) -> list[dict]:
    """
    Return list of available add-ons for the user.
    """
    # In real logic, we might filter based on eligibility
    return [dict(addon) for addon in _addon_store]

@tool
def mock_purchase_addon(user_id: str, addon_id: str) -> bool:
    """
    Purchase an add-on for the user.
    Returns True if successful, False if not found.
    """
    addon = next((a for a in _addon_store if a["id"] == addon_id), None)
    if not addon:
        return False

    _user_addon_store.append({
        "user_id": user_id,
        "addon_id": addon_id,
        "purchase_date": "2025-08-14",
        "status": "Active"
    })
    return True

@tool
def mock_ivr_identify_customer(phone: str = None, customer_no: str = None) -> dict:
    """
    Identify customer by phone number or customer number.
    Returns user profile or error if not found.
    """
    if customer_no and customer_no in _user_store_superonline:
        return dict(_user_store_superonline[customer_no])
    if phone:
        for user in _user_store_superonline.values():
            if user["phone"] == phone:
                return dict(user)
    return {"error": "Müşteri bulunamadı"}

@tool
def mock_check_service_status_superonline(customer_no: str) -> str:
    """
    Returns the service status: Active or Suspended.
    """
    user = _user_store_superonline.get(customer_no)
    if not user:
        return "Not Found"
    return user["service_status"]

@tool
def mock_check_regional_outage(city: str) -> bool:
    """
    Check if there is an outage in the customer's city.
    """
    return _regional_outage_store.get(city, False)

@tool
def mock_run_line_test(customer_no: str) -> dict:
    """
    Run automated line test for the customer's connection.
    """
    return _line_test_results.get(customer_no, {"error": "Test yapılamadı"})

@tool
def mock_reset_ip_session(customer_no: str) -> bool:
    """
    Reset IP session for the customer.
    """
    if customer_no in _line_test_results:
        _line_test_results[customer_no]["ip_session"] = "Active"
        return True
    return False

@tool
def mock_open_fault_ticket(customer_no: str, description: str) -> str:
    """
    Open a fault ticket and return the ticket number.
    """
    ticket_id = f"TCK{len(_fault_ticket_store)+1:05d}"
    _fault_ticket_store.append({
        "ticket_id": ticket_id,
        "customer_no": customer_no,
        "description": description,
        "status": "Open"
    })
    return ticket_id

@tool
def mock_get_estimated_wait_time(hour: int) -> int:
    """
    Estimate wait time based on hour of day.
    Returns minutes.
    """
    # Assume 9-12 and 17-20 are peak hours
    if 9 <= hour <= 12 or 17 <= hour <= 20:
        return 45  # minutes
    return 5

@tool
def mock_request_callback(customer_no: str) -> bool:
    """
    Record callback request for customer.
    """
    # In real case this would push to queue
    return True

@tool
def mock_power_cycle_modem(customer_no: str) -> bool:
    """
    Simulate customer turning the modem off and on.
    Returns True if modem restarted successfully.
    """
    if customer_no not in _modem_status_store:
        return False
    _modem_status_store[customer_no]["powered_on"] = True
    return True

@tool
def mock_check_modem_dsl_light(customer_no: str) -> bool:
    """
    Check if DSL/Internet light is on.
    """
    modem = _modem_status_store.get(customer_no)
    if not modem:
        return False
    return modem["dsl_light"]

# Agno Agent Yapılandırması
Bu bölümde, tüm mock fonksiyonları (tools) içeren ve değerlendirme için özelleştirilmiş Agno Agent'ı yapılandırılır. Bilgi tabanı yolu Kaggle ortamına uygun olarak güncellenir ve bellek, depolama ayarları yapılır.

In [ ]:
instructions = [
    "Soruyu cevaplamadan önce bilginizi araştır.",
    "Eğer bir veya daha fazla bilgi eksikse, herhangi bir işlem yapmadan önce müşteriden ihtiyaç duyulan bilgiyi iste.",
    "Yanıtınıza yalnızca çıktıyı ekle. Başka metin ekleme.",
]

# Agent'ın kullanacağı tüm mock araçların listesi
tools = [
    mock_get_user_info,
    mock_get_available_packages,
    mock_initiate_package_change,
    mock_get_package_features,
    mock_get_billing_history,
    mock_get_latest_invoice,
    mock_pay_invoice,
    mock_list_payment_methods,
    mock_add_payment_method,
    mock_get_usage,
    mock_open_support_ticket,
    mock_get_ticket_status,
    mock_reply_to_ticket,
    mock_cancel_subscription,
    mock_calculate_proration,
    mock_list_promotions,
    mock_apply_promo,
    mock_recommend_package,
    mock_verify_identity,
    mock_send_otp,
    mock_validate_otp,
    mock_get_outages,
    mock_upgrade_with_effective_date,
    mock_refund_last_charge,
    mock_get_latest_invoice_original, # Kalan eski fonksiyonlar
    mock_register_invoice_dispute,
    mock_check_service_status_original,
    mock_check_area_outage_original,
    mock_schedule_technician,
    mock_get_available_addons,
    mock_purchase_addon,
    mock_ivr_identify_customer,
    mock_check_service_status_superonline,
    mock_check_regional_outage,
    mock_run_line_test,
    mock_reset_ip_session,
    mock_open_fault_ticket,
    mock_get_estimated_wait_time,
    mock_request_callback,
    mock_power_cycle_modem,
    mock_check_modem_dsl_light
   # ReasoningTools(add_instructions=True) # Gerekirse muhakeme araçları eklenebilir
]

# Ollama dil modeli
model = Ollama(
    id="qwen2.5:7b-instruct",
    host="http://127.0.0.1:11434"   # Ollama’nın varsayılan REST uç noktası
)

# Ollama embedding modeli
embeddings = OllamaEmbedder(
    id="bge-m3",                     
    host="http://127.0.0.1:11434",
    dimensions=4096
)

# Bellek yapılandırması
memory = Memory(
    model=model,
    db=SqliteMemoryDb(table_name="user_memories", db_file="tmp/agent.db"),
    delete_memories=True,
    clear_memories=True,
)

# Bilgi tabanı yapılandırması (Kaggle yoluna uygun)
knowledge = MarkdownKnowledgeBase(
    path="/kaggle/input/senaryolar/senaryolar",
    vector_db=LanceDb(
        uri="tmp/lancedb",
        table_name="agno_docs",
        search_type=SearchType.hybrid,
        embedder=embeddings,
    ),
)

# Depolama yapılandırması
storage = SqliteStorage(table_name="agent_sessions", db_file="tmp/agent.db")

# Agno Agent'ının başlatılması (değerlendirme için parametreler)
agent = Agent(
    model=model,
    tools=tools,
    instructions=instructions,
    knowledge=knowledge,
    storage=storage,
    memory=memory,
    enable_agentic_memory=True,
    add_state_in_messages=True,
    add_history_to_messages=True,
    num_history_responses=5,
    markdown=True,
)

WARNING  Expected embedding dimension 4096, but got 1024

INFO Creating table: agno_docs

# Sesli Etkileşim Fonksiyonları
Bu bölüm, Google Colab ortamında kullanıcıdan ses girişi almak ve agent'ın yanıtlarını sesli olarak vermek için gerekli fonksiyonları içerir. Canlı etkileşim için kullanılır.

In [ ]:
session_id = 0
exit_session = False

def record_audio(filename="input.wav", record_seconds=5, session_callback=None):
    """
    Colab için düğme tabanlı ses kaydı.
    Dosyayı `filename` olarak kaydeder ve bittiğinde `session_callback()`'i çağırır.
    """
    if os.path.exists(filename):
        os.remove(filename)

    out = widgets.Output()
    display(out)

    def save_audio(base64data):
        header, data = base64data.split(',')
        audio_bytes = base64.b64decode(data)
        with open(filename, "wb") as f:
            f.write(audio_bytes)
        with out:
            print(f"✅ Ses {filename} olarak kaydedildi.")
        if session_callback:
            session_callback()

    colab_output.register_callback('notebook.save_audio', save_audio)

    button = widgets.Button(description="🎤 Kaydı Başlat", button_style='info')
    display(button)

    def on_click(b):
        clear_output(wait=True)
        display(out)
        display(Javascript(f"""
        navigator.mediaDevices.getUserMedia({{audio:true}})
          .then(stream => {{
            const mediaRecorder = new MediaRecorder(stream);
            const audioChunks = [];
            mediaRecorder.addEventListener("dataavailable", event => {{
              audioChunks.push(event.data);
            }});
            mediaRecorder.addEventListener("stop", () => {{
              const audioBlob = new Blob(audioChunks);
              const reader = new FileReader();
              reader.readAsDataURL(audioBlob);
              reader.onloadend = () => {{
                const base64data = reader.result;
                google.colab.kernel.invokeFunction('notebook.save_audio', [base64data], {{}});
              }};
            }});
            mediaRecorder.start();
            setTimeout(() => {{ mediaRecorder.stop(); }}, {record_seconds*1000});
          }});
        """))

    button.on_click(on_click)

def process_recording():
    """Ses kaydını işler, metne dönüştürür ve agent ile etkileşime geçer."""
    global session_id, exit_session

    try:
        spoken_text = speech_to_text("input.wav").strip()
    except Exception as e:
        print(f"❌ Konuşmadan metne dönüştürme hatası: {e}")
        if not exit_session:
            record_audio(session_callback=process_recording)
        return

    print(f"🗣️ Siz dediniz ki: {spoken_text}")

    cmd = spoken_text.lower().replace(".", "")

    if cmd == "çıkış yap":
        exit_session = True
        print("👋 Çıkış yapıldı.")
        return
    elif cmd == "yeni oturum":
        session_id += 1
        print(f"🔄 Yeni oturum başladı: {session_id}")
    else:
        if spoken_text:
            try:
                response = agent.run(
                    spoken_text,
                    stream=False,
                    debug_mode=True,
                    show_full_reasoning=True,
                    stream_intermediate_steps=True,
                    session_id=str(session_id)
                )
                display(text_to_speech(response.content))
            except Exception as e:
                print(f"Agent hatası: {e}, Metin: {spoken_text}")
        else:
            print("⚠️ Konuşma algılanmadı. Lütfen tekrar deneyin.")

    if not exit_session:
        record_audio(session_callback=process_recording)  # Bir sonraki tur için kaydı başlat

# Canlı etkileşimi başlatmak için bu fonksiyonu çalıştırın:
record_audio(session_callback=process_recording)

# Değerlendirme Ortamı ve Yardımcı Fonksiyonlar
Bu bölüm, agent'ın performansını, araç çağırma güvenilirliğini ve yanıt doğruluğunu ölçmek için kullanılan yardımcı fonksiyonları ve test veri setinin yüklenmesini içerir.

In [ ]:
# Test veri setinin temel yolu
BASE = "/kaggle/input/testset"
csv_files = glob.glob(os.path.join(BASE, "**", "*.csv"), recursive=True)
csv_path = csv_files[0]        
print("Okunuyor:", csv_path)
df = pd.read_csv(csv_path, encoding="utf-8", on_bad_lines="skip")

print(df.shape)
display(df.head())

# =============== Yardımcılar ===============

def make_perf_target(prompt: str):
    """Verilen prompt için TEK bir agent çağrısı yapıp içeriği döndürür.
    PerformanceEval içinde zamanlamak için kullanılır."""
    def perf_target():
        r = agent.run(prompt, stream=False)
        return r.content
    return perf_target

def _actual_tools_from_resp(resp) -> List[str]:
    """RunResponse içinden çağrılan araç adlarını çıkartır (hızlı yerel kontrol)."""
    tc = getattr(resp, "tool_calls", None) or []
    names = [getattr(x, "name", None) or (x.get("function", {}) or {}).get("name") for x in tc]
    return [n for n in names if n]

def run_reliability(resp, expected_calls: List[str]) -> bool:
    """ReliabilityEval ile kontrol et; None/hata durumunda yerel kontrolle devam et."""
    try:
        evaluation = ReliabilityEval(agent_response=resp, expected_tool_calls=expected_calls)
        rel_res: Optional[ReliabilityResult] = evaluation.run(print_results=False)
        if rel_res is None:
            return all(t in _actual_tools_from_resp(resp) for t in expected_calls)
        return bool(getattr(rel_res, "passed", False))
    except Exception:
        return all(t in _actual_tools_from_resp(resp) for t in expected_calls)

def run_accuracy(prompt: str, expected: str) -> Optional[float]:
    """LLM-hakem ile doğruluk puanını (0–10) hesaplar."""
    acc_eval = AccuracyEval(
        model=model,           # mevcut judge modelinizi kullanıyoruz
        agent=agent,
        input=prompt,
        expected_output=expected,
        additional_guidelines=LENIENT_GUIDELINES,
        num_iterations=1,
    )
    acc_res = acc_eval.run(print_results=False)
    return getattr(acc_res, "avg_score", None)
    
# --- Beklenen araç çağrıları (örnek) ---
# Bu kısım değerlendirilen senaryoya göre dinamik olarak belirlenmelidir.
# Genel olarak, en azından mock_get_user_info, mock_get_available_packages ve mock_initiate_package_change
# gibi temel araçların çağrıldığının kontrol edilmesi beklenebilir.
expected_calls = [
    "mock_get_user_info",
    "mock_get_available_packages",
    "mock_initiate_package_change",
]

# Ana Değerlendirme Döngüsü
Test veri setindeki her girdi için agent'ı çalıştırır ve performans (gecikme süresi), güvenilirlik (araç çağırma) ve doğruluk (LLM-as-a-judge) metriklerini toplar. Sonuçlar bir DataFrame'de saklanır ve özet raporu oluşturulur.

In [ ]:
# --- Değiştirmeyin: mevcut yönerge ---
LENIENT_GUIDELINES = "Biçimi önemseme; içerik doğruluğunu değerlendir. 0–10 arası puan ver."

# =============== Ana değerlendirme döngüsü ===============

rows = []
i = 1
for r in df[:100].itertuples(index=True):
    prompt   = str(r.user_input)
    expected = str(r.reference)

    # Orijinal akışa sadık: önce bir kez çalıştırıyoruz
    resp = agent.run(prompt, stream=False)

    # --- Performans ---\n",
    perf_eval   = PerformanceEval(func=make_perf_target(prompt), num_iterations=1, warmup_runs=0)
    perf_result = perf_eval.run(print_results=False, print_summary=False)
    latency_ms  = getattr(perf_result, "avg_run_time", None)

    # --- Güvenilirlik ---\n",
    # Not: expected_calls listesi her prompt için dinamik olarak belirlenmelidir
    # Örneğin: 'user_id=12345' içeren bir prompt için `mock_get_user_info` beklenir.
    # Şimdilik, genel bir kontrol için varsayılan `expected_calls` kullanılıyor.
    rel_pass = run_reliability(resp, expected_calls) 

    # --- Doğruluk (LLM-as-a-judge) ---\n",
    acc_score = run_accuracy(prompt, expected)

    rows.append({
        "row_id": i,
        "accuracy": acc_score,
        "latency_ms": latency_ms,
        "reliability": rel_pass,
    })
    i = i + 1

results = pd.DataFrame(rows)
summary = pd.DataFrame([{
    "n": len(results),
    "accuracy_avg": results["accuracy"].mean(),
    "latency_avg_ms": results["latency_ms"].mean(),
    "reliability_rate": results["reliability"].mean(),
}])

print("=== ÖZET ===")
display(summary)

# Değerlendirme Sonuçlarını Kaydetme
Elde edilen detaylı sonuçlar ve özet istatistikler, gelecekteki analizler ve raporlama için CSV dosyaları olarak kaydedilir.

In [ ]:
out_dir = "/kaggle/working"
os.makedirs(out_dir, exist_ok=True)

results.to_csv(os.path.join(out_dir, "agent_eval_results.csv"), index=False, encoding="utf-8")
summary.to_csv(os.path.join(out_dir, "agent_eval_summary.csv"), index=False, encoding="utf-8")

print("Kaydedildi:",
      os.path.join(out_dir, "agent_eval_results.csv"),
      "ve",
      os.path.join(out_dir, "agent_eval_summary.csv"))

# Arayüz

In [ ]:
# Colab'da Çalışan Birleşik AI Asistanı Arayüzü (Metin + Ses)
# Bu kodu notebook'unuzda ayrı bir hücreye yapıştırın

from IPython.display import HTML, display, Javascript, clear_output
from google.colab import output
import json
import base64
import uuid
import os
import ipywidgets as widgets

# Global değişkenler
interface_session_id = 1
interface_messages = []
session_id = 0
exit_session = False

def create_ai_interface():
    """AI Arayüzünü oluştur ve görüntüle"""
    
    # CSS ve HTML içeriği
    interface_html = """
    <div id="ai-interface-container" style="
        width: 100%;
        height: 700px;
        border: 2px solid #4f46e5;
        border-radius: 15px;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        display: flex;
        flex-direction: column;
        overflow: hidden;
        font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif;
        position: relative;
    ">
        <!-- Header -->
        <div style="
            background: linear-gradient(135deg, #4f46e5 0%, #7c3aed 100%);
            color: white;
            padding: 15px 20px;
            display: flex;
            justify-content: space-between;
            align-items: center;
        ">
            <h3 style="margin: 0; font-size: 18px; font-weight: 700;">
                🤖 AI Müşteri Hizmetleri Asistanı
            </h3>
            <div style="display: flex; align-items: center; gap: 15px;">
                <div style="
                    display: flex;
                    align-items: center;
                    gap: 8px;
                    font-size: 12px;
                    background: rgba(255, 255, 255, 0.1);
                    padding: 5px 12px;
                    border-radius: 15px;
                ">
                    <div style="
                        width: 8px;
                        height: 8px;
                        background: #10b981;
                        border-radius: 50%;
                        animation: pulse 2s infinite;
                    "></div>
                    Çevrimiçi
                </div>
                <button onclick="toggleVoiceMode()" id="voice-mode-btn" style="
                    background: rgba(255, 255, 255, 0.1);
                    color: white;
                    border: 1px solid rgba(255, 255, 255, 0.2);
                    padding: 8px 12px;
                    border-radius: 15px;
                    cursor: pointer;
                    font-size: 12px;
                    transition: all 0.3s ease;
                " onmouseover="this.style.background='rgba(255, 255, 255, 0.2)'" onmouseout="this.style.background='rgba(255, 255, 255, 0.1)'">
                    🎤 Ses Modu
                </button>
            </div>
        </div>

        <!-- Main Content -->
        <div style="
            display: flex;
            flex: 1;
            background: white;
        ">
            <!-- Sidebar -->
            <div style="
                width: 250px;
                background: #f8fafc;
                border-right: 1px solid #e2e8f0;
                padding: 15px;
                overflow-y: auto;
            ">
                <div style="margin-bottom: 15px;">
                    <h4 style="
                        font-size: 12px;
                        font-weight: 600;
                        color: #1e293b;
                        margin: 0 0 10px 0;
                        text-transform: uppercase;
                        letter-spacing: 0.5px;
                    ">Kullanıcı Bilgileri</h4>
                    <div style="
                        background: white;
                        padding: 12px;
                        border-radius: 8px;
                        border: 1px solid #e2e8f0;
                    ">
                        <div style="color: #1e293b; font-weight: 600; margin-bottom: 5px;">Misafir Kullanıcı</div>
                        <div style="color: #64748b; font-size: 12px; margin-bottom: 3px;">Paket: Belirlenmedi</div>
                        <div style="color: #64748b; font-size: 12px;" id="session-info">Oturum: 1</div>
                    </div>
                </div>

                <div style="margin-bottom: 15px;">
                    <h4 style="
                        font-size: 12px;
                        font-weight: 600;
                        color: #1e293b;
                        margin: 0 0 10px 0;
                        text-transform: uppercase;
                        letter-spacing: 0.5px;
                    ">Ses Kontrolleri</h4>
                    <div style="display: flex; flex-direction: column; gap: 6px;">
                        <button onclick="startVoiceRecording()" id="record-btn" style="
                            padding: 10px 12px;
                            background: #10b981;
                            color: white;
                            border: none;
                            border-radius: 6px;
                            cursor: pointer;
                            font-size: 12px;
                            text-align: left;
                            transition: all 0.2s ease;
                        " onmouseover="this.style.background='#059669'" onmouseout="this.style.background='#10b981'">
                            🎤 Ses Kaydı Başlat
                        </button>
                        <button onclick="newSession()" style="
                            padding: 10px 12px;
                            background: white;
                            border: 1px solid #e2e8f0;
                            border-radius: 6px;
                            cursor: pointer;
                            font-size: 12px;
                            text-align: left;
                            transition: all 0.2s ease;
                        " onmouseover="this.style.background='#f1f5f9'" onmouseout="this.style.background='white'">
                            🔄 Yeni Oturum
                        </button>
                    </div>
                </div>

                <div>
                    <h4 style="
                        font-size: 12px;
                        font-weight: 600;
                        color: #1e293b;
                        margin: 0 0 10px 0;
                        text-transform: uppercase;
                        letter-spacing: 0.5px;
                    ">Hızlı İşlemler</h4>
                    <div style="display: flex; flex-direction: column; gap: 6px;">
                        <button onclick="sendQuickMessage('Kullanıcı ID: 12345. Paket değiştirmek istiyorum.')" style="
                            padding: 10px 12px;
                            background: white;
                            border: 1px solid #e2e8f0;
                            border-radius: 6px;
                            cursor: pointer;
                            font-size: 12px;
                            text-align: left;
                            transition: all 0.2s ease;
                        " onmouseover="this.style.background='#f1f5f9'" onmouseout="this.style.background='white'">
                            📦 Paket Değiştir
                        </button>
                        <button onclick="sendQuickMessage('Kullanıcı ID: 12345. Son faturamı görmek istiyorum.')" style="
                            padding: 10px 12px;
                            background: white;
                            border: 1px solid #e2e8f0;
                            border-radius: 6px;
                            cursor: pointer;
                            font-size: 12px;
                            text-align: left;
                            transition: all 0.2s ease;
                        " onmouseover="this.style.background='#f1f5f9'" onmouseout="this.style.background='white'">
                            💰 Fatura Sorgula
                        </button>
                        <button onclick="sendQuickMessage('Kullanıcı ID: 12345. Kullanım durumumu öğrenmek istiyorum.')" style="
                            padding: 10px 12px;
                            background: white;
                            border: 1px solid #e2e8f0;
                            border-radius: 6px;
                            cursor: pointer;
                            font-size: 12px;
                            text-align: left;
                            transition: all 0.2s ease;
                        " onmouseover="this.style.background='#f1f5f9'" onmouseout="this.style.background='white'">
                            📊 Kullanım Durumu
                        </button>
                        <button onclick="sendQuickMessage('Kullanıcı ID: 12345. İnternet bağlantımda sorun var.')" style="
                            padding: 10px 12px;
                            background: white;
                            border: 1px solid #e2e8f0;
                            border-radius: 6px;
                            cursor: pointer;
                            font-size: 12px;
                            text-align: left;
                            transition: all 0.2s ease;
                        " onmouseover="this.style.background='#f1f5f9'" onmouseout="this.style.background='white'">
                            🌐 Teknik Destek
                        </button>
                    </div>
                </div>
            </div>

            <!-- Chat Area -->
            <div style="
                flex: 1;
                display: flex;
                flex-direction: column;
            ">
                <!-- Messages -->
                <div id="messages-container" style="
                    flex: 1;
                    padding: 20px;
                    overflow-y: auto;
                    display: flex;
                    flex-direction: column;
                    gap: 15px;
                ">
                    <!-- Welcome Message -->
                    <div style="
                        display: flex;
                        gap: 10px;
                        max-width: 80%;
                    ">
                        <div style="
                            width: 35px;
                            height: 35px;
                            border-radius: 50%;
                            background: linear-gradient(135deg, #10b981, #047857);
                            color: white;
                            display: flex;
                            align-items: center;
                            justify-content: center;
                            font-size: 12px;
                            flex-shrink: 0;
                        ">🤖</div>
                        <div style="
                            background: #f8fafc;
                            padding: 12px 16px;
                            border-radius: 15px;
                            border: 1px solid #e2e8f0;
                            max-width: 100%;
                            word-wrap: break-word;
                            font-size: 14px;
                            line-height: 1.4;
                        ">
                            Merhaba! Size nasıl yardımcı olabilirim? Paket değişikliği, fatura sorgulama, teknik destek gibi konularda yardım edebilirim. Metin yazabilir veya 🎤 Ses Kaydı Başlat butonuna tıklayarak konuşabilirsiniz. Lütfen kullanıcı ID'nizi belirtin veya hızlı işlemlerden birini seçin.
                        </div>
                    </div>
                </div>

                <!-- Recording Status -->
                <div id="recording-status" style="
                    padding: 10px 20px;
                    background: #fef3c7;
                    border-top: 1px solid #fbbf24;
                    display: none;
                    align-items: center;
                    gap: 10px;
                ">
                    <div style="
                        width: 12px;
                        height: 12px;
                        background: #ef4444;
                        border-radius: 50%;
                        animation: pulse 1s infinite;
                    "></div>
                    <span style="font-size: 14px; color: #92400e;">🎤 Ses kaydediliyor... (5 saniye)</span>
                </div>

                <!-- Input Area -->
                <div style="
                    padding: 15px 20px;
                    background: white;
                    border-top: 1px solid #e2e8f0;
                    display: flex;
                    gap: 10px;
                    align-items: center;
                ">
                    <input 
                        type="text" 
                        id="message-input" 
                        placeholder="Mesajınızı yazın..." 
                        style="
                            flex: 1;
                            padding: 12px 16px;
                            border: 2px solid #e2e8f0;
                            border-radius: 20px;
                            font-size: 14px;
                            outline: none;
                            background: #f8fafc;
                            transition: all 0.3s ease;
                        "
                        onkeypress="if(event.key==='Enter') sendMessage()"
                        onfocus="this.style.borderColor='#4f46e5'; this.style.background='white'"
                        onblur="this.style.borderColor='#e2e8f0'; this.style.background='#f8fafc'"
                    >
                    <button 
                        onclick="sendMessage()" 
                        style="
                            padding: 12px 16px;
                            background: linear-gradient(135deg, #4f46e5, #7c3aed);
                            color: white;
                            border: none;
                            border-radius: 50%;
                            cursor: pointer;
                            font-size: 14px;
                            width: 45px;
                            height: 45px;
                            display: flex;
                            align-items: center;
                            justify-content: center;
                            transition: all 0.3s ease;
                        "
                        onmouseover="this.style.transform='translateY(-2px)'; this.style.boxShadow='0 4px 12px rgba(0,0,0,0.15)'"
                        onmouseout="this.style.transform='translateY(0)'; this.style.boxShadow='none'"
                    >
                        ➤
                    </button>
                </div>
            </div>
        </div>
    </div>

    <style>
        @keyframes pulse {
            0%, 100% { opacity: 1; }
            50% { opacity: 0.5; }
        }
        
        #messages-container::-webkit-scrollbar {
            width: 6px;
        }
        
        #messages-container::-webkit-scrollbar-track {
            background: #f1f1f1;
            border-radius: 3px;
        }
        
        #messages-container::-webkit-scrollbar-thumb {
            background: #c1c1c1;
            border-radius: 3px;
        }
        
        #messages-container::-webkit-scrollbar-thumb:hover {
            background: #a1a1a1;
        }
        
        .message-animation {
            animation: slideIn 0.3s ease-out;
        }
        
        @keyframes slideIn {
            from {
                opacity: 0;
                transform: translateY(20px);
            }
            to {
                opacity: 1;
                transform: translateY(0);
            }
        }
    </style>

    <script>
        let isProcessing = false;
        let isRecording = false;
        let voiceModeActive = false;
        let currentSessionId = 1;
        
        function toggleVoiceMode() {
            voiceModeActive = !voiceModeActive;
            const btn = document.getElementById('voice-mode-btn');
            if (voiceModeActive) {
                btn.textContent = '💬 Metin Modu';
                btn.style.background = 'rgba(34, 197, 94, 0.2)';
            } else {
                btn.textContent = '🎤 Ses Modu';
                btn.style.background = 'rgba(255, 255, 255, 0.1)';
            }
        }
        
        function newSession() {
            currentSessionId++;
            document.getElementById('session-info').textContent = 'Oturum: ' + currentSessionId;
            addMessage('🔄 Yeni oturum başladı: ' + currentSessionId, 'system');
            
            // Python'a yeni oturum bilgisini gönder
            google.colab.kernel.invokeFunction('update_session_id', [currentSessionId], {});
        }
        
        function startVoiceRecording() {
            if (isRecording || isProcessing) return;
            
            isRecording = true;
            const recordBtn = document.getElementById('record-btn');
            const recordingStatus = document.getElementById('recording-status');
            
            recordBtn.textContent = '⏹ Kayıt Durduruluyor...';
            recordBtn.style.background = '#ef4444';
            recordingStatus.style.display = 'flex';
            
            // Ses kaydını başlat
            google.colab.kernel.invokeFunction('start_voice_recording', [], {});
            
            // 5 saniye sonra kayıt bitir
            setTimeout(() => {
                recordBtn.textContent = '🎤 Ses Kaydı Başlat';
                recordBtn.style.background = '#10b981';
                recordingStatus.style.display = 'none';
                isRecording = false;
            }, 5000);
        }
        
        function sendQuickMessage(message) {
            document.getElementById('message-input').value = message;
            sendMessage();
        }
        
        async function sendMessage() {
            const input = document.getElementById('message-input');
            const message = input.value.trim();
            
            if (!message || isProcessing) return;
            
            input.value = '';
            isProcessing = true;
            
            // Add user message
            addMessage(message, 'user');
            
            // Add loading message
            const loadingId = addLoadingMessage();
            
            try {
                // Call Python backend
                google.colab.kernel.invokeFunction('process_ai_message', [message], {});
                
            } catch (error) {
                console.error('Error:', error);
                removeLoadingMessage(loadingId);
                addMessage('Bir hata oluştu. Lütfen tekrar deneyin.', 'assistant');
                isProcessing = false;
            }
        }
        
        function addMessage(content, sender) {
            const container = document.getElementById('messages-container');
            const messageDiv = document.createElement('div');
            
            const isUser = sender === 'user';
            const isSystem = sender === 'system';
            let avatar, bgColor, textColor, alignment, avatarBg;
            
            if (isSystem) {
                avatar = '⚡';
                bgColor = '#fef3c7';
                textColor = '#92400e';
                alignment = 'center';
                avatarBg = 'linear-gradient(135deg, #f59e0b, #d97706)';
                messageDiv.style.cssText = `
                    display: flex;
                    justify-content: center;
                    margin: 10px 0;
                `;
                messageDiv.innerHTML = `
                    <div style="
                        background: ${bgColor};
                        color: ${textColor};
                        padding: 8px 16px;
                        border-radius: 15px;
                        border: 1px solid #fbbf24;
                        font-size: 12px;
                        font-weight: 500;
                    " class="message-animation">
                        ${content}
                    </div>
                `;
            } else {
                avatar = isUser ? '👤' : '🤖';
                bgColor = isUser ? 'linear-gradient(135deg, #3b82f6, #1d4ed8)' : '#f8fafc';
                textColor = isUser ? 'white' : '#1e293b';
                alignment = isUser ? 'flex-end' : 'flex-start';
                avatarBg = isUser ? 'linear-gradient(135deg, #3b82f6, #1d4ed8)' : 'linear-gradient(135deg, #10b981, #047857)';
                
                messageDiv.style.cssText = `
                    display: flex;
                    gap: 10px;
                    max-width: 80%;
                    align-self: ${alignment};
                    ${isUser ? 'flex-direction: row-reverse;' : ''}
                `;
                
                messageDiv.innerHTML = `
                    <div style="
                        width: 35px;
                        height: 35px;
                        border-radius: 50%;
                        background: ${avatarBg};
                        color: white;
                        display: flex;
                        align-items: center;
                        justify-content: center;
                        font-size: 12px;
                        flex-shrink: 0;
                    ">${avatar}</div>
                    <div style="
                        background: ${bgColor};
                        color: ${textColor};
                        padding: 12px 16px;
                        border-radius: 15px;
                        border: 1px solid ${isUser ? 'transparent' : '#e2e8f0'};
                        max-width: 100%;
                        word-wrap: break-word;
                        font-size: 14px;
                        line-height: 1.4;
                    " class="message-animation">
                        ${content}
                    </div>
                `;
            }
            
            container.appendChild(messageDiv);
            container.scrollTop = container.scrollHeight;
        }
        
        function addLoadingMessage() {
            const container = document.getElementById('messages-container');
            const messageDiv = document.createElement('div');
            const loadingId = 'loading-' + Date.now();
            messageDiv.id = loadingId;
            
            messageDiv.style.cssText = `
                display: flex;
                gap: 10px;
                max-width: 80%;
                align-self: flex-start;
            `;
            
            messageDiv.innerHTML = `
                <div style="
                    width: 35px;
                    height: 35px;
                    border-radius: 50%;
                    background: linear-gradient(135deg, #10b981, #047857);
                    color: white;
                    display: flex;
                    align-items: center;
                    justify-content: center;
                    font-size: 12px;
                    flex-shrink: 0;
                ">🤖</div>
                <div style="
                    background: #f8fafc;
                    padding: 12px 16px;
                    border-radius: 15px;
                    border: 1px solid #e2e8f0;
                    display: flex;
                    gap: 4px;
                    align-items: center;
                ">
                    <div style="
                        width: 8px;
                        height: 8px;
                        background: #94a3b8;
                        border-radius: 50%;
                        animation: bounce 1.4s infinite ease-in-out both;
                    "></div>
                    <div style="
                        width: 8px;
                        height: 8px;
                        background: #94a3b8;
                        border-radius: 50%;
                        animation: bounce 1.4s infinite ease-in-out both;
                        animation-delay: -0.16s;
                    "></div>
                    <div style="
                        width: 8px;
                        height: 8px;
                        background: #94a3b8;
                        border-radius: 50%;
                        animation: bounce 1.4s infinite ease-in-out both;
                        animation-delay: -0.32s;
                    "></div>
                </div>
            `;
            
            // Add bounce animation
            if (!document.getElementById('bounce-style')) {
                const style = document.createElement('style');
                style.id = 'bounce-style';
                style.textContent = `
                    @keyframes bounce {
                        0%, 80%, 100% { transform: scale(0); }
                        40% { transform: scale(1); }
                    }
                `;
                document.head.appendChild(style);
            }
            
            container.appendChild(messageDiv);
            container.scrollTop = container.scrollHeight;
            
            return loadingId;
        }
        
        function removeLoadingMessage(loadingId) {
            const loadingElement = document.getElementById(loadingId);
            if (loadingElement) {
                loadingElement.remove();
            }
        }
        
        // Global functions to handle Python responses
        window.handleAIResponse = function(response, error, isVoice = false) {
            // Remove loading message
            const loadingElements = document.querySelectorAll('[id^="loading-"]');
            loadingElements.forEach(el => el.remove());
            
            if (error) {
                addMessage('Üzgünüm, bir hata oluştu: ' + error, 'assistant');
            } else {
                addMessage(response, 'assistant');
                if (isVoice) {
                    addMessage('🔊 Ses yanıtı çalınıyor...', 'system');
                }
            }
            
            isProcessing = false;
        };
        
        window.handleVoiceInput = function(spokenText, error) {
            if (error) {
                addMessage('❌ Ses tanıma hatası: ' + error, 'system');
                isProcessing = false;
                return;
            }
            
            if (spokenText) {
                addMessage(spokenText + ' 🎤', 'user');
                
                // Özel komutları kontrol et
                const cmd = spokenText.toLowerCase().replace(".", "");
                if (cmd === "çıkış yap") {
                    addMessage('👋 Çıkış yapıldı.', 'system');
                    return;
                } else if (cmd === "yeni oturum") {
                    newSession();
                    return;
                }
                
                // Add loading message for voice processing
                const loadingId = addLoadingMessage();
                
                try {
                    // Call Python backend for voice message
                    google.colab.kernel.invokeFunction('process_voice_message', [spokenText], {});
                } catch (error) {
                    console.error('Voice processing error:', error);
                    removeLoadingMessage(loadingId);
                    addMessage('Ses işleme sırasında bir hata oluştu.', 'assistant');
                    isProcessing = false;
                }
            } else {
                addMessage('⚠ Konuşma algılanmadı. Lütfen tekrar deneyin.', 'system');
                isProcessing = false;
            }
        };
    </script>
    """
    
    display(HTML(interface_html))

# Python callback fonksiyonları

def update_session_id(new_session_id):
    """Oturum ID'sini güncelle"""
    global interface_session_id, session_id
    interface_session_id = new_session_id
    session_id = new_session_id
    print(f"🔄 Oturum güncellendi: {session_id}")

def start_voice_recording():
    """Ses kaydını başlat"""
    print("🎤 Ses kaydı başlatılıyor...")
    record_audio_for_interface()

def process_ai_message(message):
    """AI mesajını işle (metin)"""
    try:
        print(f"📝 İşlenen mesaj: {message}")
        
        # Agent'ı çalıştır (mevcut agent'ınızı kullanın)
        response = agent.run(
            message,
            stream=False,
            session_id=str(interface_session_id)
        )
        
        # Yanıtı JavaScript'e gönder
        display(Javascript(f"""
            if (window.handleAIResponse) {{
                window.handleAIResponse({json.dumps(response.content)}, null, false);
            }}
        """))
        
    except Exception as e:
        print(f"❌ Agent hatası: {e}")
        # Hata durumunda JavaScript'e hata gönder
        display(Javascript(f"""
            if (window.handleAIResponse) {{
                window.handleAIResponse(null, {json.dumps(str(e))}, false);
            }}
        """))

def process_voice_message(spoken_text):
    """Ses mesajını işle"""
    try:
        print(f"🗣 Ses mesajı işleniyor: {spoken_text}")
        
        # Agent'ı çalıştır
        response = agent.run(
            spoken_text,
            stream=False,
            debug_mode=True,
            show_full_reasoning=True,
            stream_intermediate_steps=True,
            session_id=str(interface_session_id)
        )
        
        # Ses yanıtı oluştur
        try:
            display(text_to_speech(response.content))
            # JavaScript'e yanıt gönder (ses ile)
            display(Javascript(f"""
                if (window.handleAIResponse) {{
                    window.handleAIResponse({json.dumps(response.content)}, null, true);
                }}
            """))
        except Exception as tts_error:
            print(f"TTS hatası: {tts_error}")
            # Ses hatası olsa bile metin yanıtı gönder
            display(Javascript(f"""
                if (window.handleAIResponse) {{
                    window.handleAIResponse({json.dumps(response.content)}, null, false);
                }}
            """))
        
    except Exception as e:
        print(f"❌ Ses mesajı agent hatası: {e}")
        display(Javascript(f"""
            if (window.handleAIResponse) {{
                window.handleAIResponse(null, {json.dumps(str(e))}, false);
            }}
        """))

def record_audio_for_interface(filename="input.wav", record_seconds=5):
    """
    Arayüz için ses kaydı fonksiyonu
    """
    if os.path.exists(filename):
        os.remove(filename)

    def save_audio(base64data):
        header, data = base64data.split(',')
        audio_bytes = base64.b64decode(data)
        with open(filename, "wb") as f:
            f.write(audio_bytes)
        print(f"✅ Ses {filename} olarak kaydedildi.")
        # Kayıt tamamlandıktan sonra işle
        process_audio_recording(filename)

    output.register_callback('notebook.save_audio_interface', save_audio)

    display(Javascript(f"""
        navigator.mediaDevices.getUserMedia({{audio:true}})
          .then(stream => {{
            const mediaRecorder = new MediaRecorder(stream);
            const audioChunks = [];
            mediaRecorder.addEventListener("dataavailable", event => {{
              audioChunks.push(event.data);
            }});
            mediaRecorder.addEventListener("stop", () => {{
              const audioBlob = new Blob(audioChunks);
              const reader = new FileReader();
              reader.readAsDataURL(audioBlob);
              reader.onloadend = () => {{
                const base64data = reader.result;
                google.colab.kernel.invokeFunction('notebook.save_audio_interface', [base64data], {{}});
              }};
            }});
            mediaRecorder.start();
            setTimeout(() => {{ mediaRecorder.stop(); }}, {record_seconds*1000});
          }});
    """))

def process_audio_recording(filename="input.wav"):
    """Ses kaydını işler ve JavaScript'e sonucu gönderir"""
    try:
        spoken_text = speech_to_text(filename).strip()
        print(f"🗣 Algılanan metin: {spoken_text}")
        
        # JavaScript'e konuşma metnini gönder
        display(Javascript(f"""
            if (window.handleVoiceInput) {{
                window.handleVoiceInput({json.dumps(spoken_text)}, null);
            }}
        """))
        
    except Exception as e:
        print(f"❌ Konuşmadan metne dönüştürme hatası: {e}")
        display(Javascript(f"""
            if (window.handleVoiceInput) {{
                window.handleVoiceInput(null, {json.dumps(str(e))});
            }}
        """))

# Ses tabanlı etkileşim için bağımsız fonksiyonlar
def record_audio(filename="input.wav", record_seconds=5, session_callback=None):
    """
    Bağımsız ses kaydı (eski yöntem için)
    """
    if os.path.exists(filename):
        os.remove(filename)

    out = widgets.Output()
    display(out)

    def save_audio(base64data):
        header, data = base64data.split(',')
        audio_bytes = base64.b64decode(data)
        with open(filename, "wb") as f:
            f.write(audio_bytes)
        with out:
            print(f"✅ Ses {filename} olarak kaydedildi.")
        if session_callback:
            session_callback()

    output.register_callback('notebook.save_audio', save_audio)

    button = widgets.Button(description="🎤 Kaydı Başlat", button_style='info')
    display(button)

    def on_click(b):
        clear_output(wait=True)
        display(out)
        display(Javascript(f"""
        navigator.mediaDevices.getUserMedia({{audio:true}})
          .then(stream => {{
            const mediaRecorder = new MediaRecorder(stream);
            const audioChunks = [];
            mediaRecorder.addEventListener("dataavailable", event => {{
              audioChunks.push(event.data);
            }});
            mediaRecorder.addEventListener("stop", () => {{
              const audioBlob = new Blob(audioChunks);
              const reader = new FileReader();
              reader.readAsDataURL(audioBlob);
              reader.onloadend = () => {{
                const base64data = reader.result;
                google.colab.kernel.invokeFunction('notebook.save_audio', [base64data], {{}});
              }};
            }});
            mediaRecorder.start();
            setTimeout(() => {{ mediaRecorder.stop(); }}, {record_seconds*1000});
          }});
        """))

    button.on_click(on_click)

def process_recording():
    """Bağımsız ses kaydını işler (eski yöntem için)"""
    global session_id, exit_session

    try:
        spoken_text = speech_to_text("input.wav").strip()
    except Exception as e:
        print(f"❌ Konuşmadan metne dönüştürme hatası: {e}")
        if not exit_session:
            record_audio(session_callback=process_recording)
        return

    print(f"🗣 Siz dediniz ki: {spoken_text}")

    cmd = spoken_text.lower().replace(".", "")

    if cmd == "çıkış yap":
        exit_session = True
        print("👋 Çıkış yapıldı.")
        return
    elif cmd == "yeni oturum":
        session_id += 1
        print(f"🔄 Yeni oturum başladı: {session_id}")
    else:
        if spoken_text:
            try:
                response = agent.run(
                    spoken_text,
                    stream=False,
                    debug_mode=True,
                    show_full_reasoning=True,
                    stream_intermediate_steps=True,
                    session_id=str(session_id)
                )
                display(text_to_speech(response.content))
            except Exception as e:
                print(f"Agent hatası: {e}, Metin: {spoken_text}")
        else:
            print("⚠ Konuşma algılanmadı. Lütfen tekrar deneyin.")

    if not exit_session:
        record_audio(session_callback=process_recording)

# Callback'leri kaydet
output.register_callback('process_ai_message', process_ai_message)
output.register_callback('process_voice_message', process_voice_message)
output.register_callback('update_session_id', update_session_id)
output.register_callback('start_voice_recording', start_voice_recording)

# Ana arayüz başlatma fonksiyonu
def start_unified_ai_interface():
    """Birleşik AI arayüzünü başlat"""
    clear_output()
    print("🚀 Birleşik AI Asistanı (Metin + Ses) başlatılıyor...")
    print("📱 Arayüz yükleniyor...")
    print("🎤 Ses özellikleri aktifleştiriliyor...")
    create_ai_interface()
    print("✅ Arayüz hazır! Yukarıdaki arayüzü kullanabilirsiniz.")
    print()
    print("💡 Özellikler:")
    print("   • Metin tabanlı sohbet")
    print("   • Ses kaydı ve tanıma")
    print("   • Sesli yanıtlar")
    print("   • Hızlı işlemler")
    print("   • Oturum yönetimi")
    print()
    print("🎯 Kullanım:")
    print("   1. Metin kutusuna yazıp Enter'a basın")
    print("   2. 'Ses Kaydı Başlat' butonuna tıklayın")
    print("   3. Hızlı işlemler butonlarını kullanın")
    print("   4. 'Yeni Oturum' ile sıfırlayın")

def start_voice_only_interface():
    """Sadece ses tabanlı arayüz (bağımsız)"""
    clear_output()
    print("🎤 Sadece Ses Tabanlı AI Asistanı başlatılıyor...")
    print("📢 Komutlar:")
    print("   • 'Çıkış yap' - Programı sonlandır")
    print("   • 'Yeni oturum' - Yeni bir oturum başlat")
    print()
    record_audio(session_callback=process_recording)

# Arayüzü başlatma seçenekleri
print("🤖 AI Asistanı Arayüz Seçenekleri:")
print()
print("1. Birleşik arayüz için:")
print("   start_unified_ai_interface()")
print()
print("2. Sadece ses arayüzü için:")
print("   start_voice_only_interface()")
print()
print("👆 Yukarıdaki komutlardan birini çalıştırarak başlayın!")

# Otomatik olarak birleşik arayüzü başlat
start_unified_ai_interface()